In [ ]:
import csv
import random
import nltk 
from nltk.tokenize import word_tokenize
import datetime
import time
import random
import itertools
import numpy as np
import pickle
from numpy.linalg import cholesky
from keras.utils.np_utils import *

In [ ]:
def newsample(nnn,ratio):
    if ratio >len(nnn):
        return random.sample(nnn*(ratio//len(nnn)+1),ratio)
    else:
        return random.sample(nnn,ratio)

In [ ]:
def preprocess_user_file(file='ClickData4.tsv',npratio=4):
    userid_dict={}
    with open(file) as f:
        userdata=f.readlines()
    for user in userdata:
        line=user.strip().split('\t')
        userid=line[0]
        if userid not in userid_dict:
            userid_dict[userid]=len(userid_dict)
    
    all_train_id=[]
    all_train_pn=[]    
    all_label=[]
    
    all_test_id=[]
    all_test_pn=[]    
    all_test_label=[]
    all_test_index=[]
    
    all_user_pos=[]
    all_test_user_pos=[]
    
    for user in userdata:
        line=user.strip().split('\t')
        userid=line[0]
        if len(line)==4:
            
            impre=[x.split('#TAB#') for x in line[2].split('#N#')]
        if len(line)==3:
            impre=[x.split('#TAB#') for x in line[2].split('#N#')]
    
        trainpos=[x[0].split() for x in impre]
        trainneg=[x[1].split() for x in impre]
         
        poslist=list(itertools.chain(*(trainpos)))
        neglist=list(itertools.chain(*(trainneg)))
    
        
        if len(line)==4:
            testimpre=[x.split('#TAB#') for x in line[3].split('#N#')]
            testpos=[x[0].split() for x in testimpre]
            testneg=[x[1].split() for x in testimpre]
            
            
            for i in range(len(testpos)):
                sess_index=[]
                sess_index.append(len(all_test_pn))
                posset=list(set(poslist))
                allpos=[int(p) for p in random.sample(posset,min(50,len(posset)))[:50]]
                allpos+=[0]*(50-len(allpos))
        
                
                for j in testpos[i]:
                    all_test_pn.append(int(j))
                    all_test_label.append(1)
                    all_test_id.append(userid_dict[userid])
                    all_test_user_pos.append(allpos)
                    
                for j in testneg[i]:
                    all_test_pn.append(int(j))
                    all_test_label.append(0)
                    all_test_id.append(userid_dict[userid])
                    all_test_user_pos.append(allpos)
                sess_index.append(len(all_test_pn))
                all_test_index.append(sess_index)
                
    
    
                
        for impre_id in range(len(trainpos)):
            for pos_sample in trainpos[impre_id]:
    
                pos_neg_sample=newsample(trainneg[impre_id],npratio)
                pos_neg_sample.append(pos_sample)
                temp_label=[0]*npratio+[1]
                temp_id=list(range(npratio+1))
                random.shuffle(temp_id)
    
                
                shuffle_sample=[]
                shuffle_label=[]
                for id in temp_id:
                    shuffle_sample.append(int(pos_neg_sample[id]))
                    shuffle_label.append(temp_label[id])
                
                posset=list(set(poslist)-set([pos_sample]))
                allpos=[int(p) for p in random.sample(posset,min(50,len(posset)))[:50]]
                allpos+=[0]*(50-len(allpos))
                all_train_pn.append(shuffle_sample)
                all_label.append(shuffle_label)
                all_train_id.append(userid_dict[userid])
                all_user_pos.append(allpos)
            
    all_train_pn=np.array(all_train_pn,dtype='int32')
    all_label=np.array(all_label,dtype='int32')
    all_train_id=np.array(all_train_id,dtype='int32')
    all_test_pn=np.array(all_test_pn,dtype='int32')
    all_test_label=np.array(all_test_label,dtype='int32')
    all_test_id=np.array(all_test_id,dtype='int32')
    all_user_pos=np.array(all_user_pos,dtype='int32')
    all_test_user_pos=np.array(all_test_user_pos,dtype='int32')
    return userid_dict,all_train_pn,all_label,all_train_id,all_test_pn,all_test_label,all_test_id,all_user_pos,all_test_user_pos

In [ ]:
def preprocess_news_file(file='DocMeta3.tsv'):
    with open(file) as f:
        newsdata=f.readlines()
    
    news={}
    category={'None':0}
    subcategory={'None':0}

    for newsline in newsdata:
        line=newsline.strip().split('\t')
        news[line[1]]=[line[2],line[3],word_tokenize(line[6].lower()),word_tokenize(line[7].lower())]
        if line[2] not in category:
            category[line[2]]=len(category)
        if line[3] not in subcategory:
            subcategory[line[3]]=len(subcategory)
            
    word_dict_raw={'PADDING':[0,999999]}
    
    for docid in news:
        for word in news[docid][2]:
            if word in word_dict_raw:
                word_dict_raw[word][1]+=1
            else:
                word_dict_raw[word]=[len(word_dict_raw),1]
        for word in news[docid][3]:
            if word in word_dict_raw:
                word_dict_raw[word][1]+=1
            else:
                word_dict_raw[word]=[len(word_dict_raw),1]
    word_dict={}
    for i in word_dict_raw:
        if word_dict_raw[i][1]>=3:
            word_dict[i]=[len(word_dict),word_dict_raw[i][1]]
    print(len(word_dict),len(word_dict_raw))
    
    news_words=[[0]*30]
    news_index={'0':0}
    for newsid in news:
        word_id=[]
        news_index[newsid]=len(news_index)
        for word in news[newsid][2]:
            if word in word_dict:
                word_id.append(word_dict[word][0])
        word_id=word_id[:30]
        news_words.append(word_id+[0]*(30-len(word_id)))
    news_words=np.array(news_words,dtype='int32') 
    
    news_body=[[0]*300]
    for newsid in news:
        word_id=[]
        for word in news[newsid][3]:
            if word in word_dict:
                word_id.append(word_dict[word][0])
        word_id=word_id[:300]
        news_body.append(word_id+[0]*(300-len(word_id)))
    news_body=np.array(news_body,dtype='int32')     
    news_v=[[0]]
    news_sv=[[0]]
    for newsid in news:
        news_v.append([category[news[newsid][0]]])
    for newsid in news:
        news_sv.append([subcategory[news[newsid][1]]])
    news_v=np.array(news_v,dtype='int32') 
    news_sv=np.array(news_sv,dtype='int32')     
    return word_dict,category,subcategory,news_words,news_body,news_v,news_sv,news_index

In [ ]:
def get_embedding(word_dict):
    embedding_dict={}
    cnt=0
    with open('/data/wuch/glove.840B.300d.txt','rb')as f:
        linenb=0
        while True:
            line=f.readline()
            if len(line)==0:
                break
            line = line.split()
            word=line[0].decode()
            linenb+=1
            if len(word) != 0:
                vec=[float(x) for x in line[1:]]
                if word in word_dict:
                    embedding_dict[word]=vec
                    if cnt%1000==0:
                        print(cnt,linenb,word)
                    cnt+=1

    embedding_matrix=[0]*len(word_dict)
    cand=[]
    for i in embedding_dict:
        embedding_matrix[word_dict[i][0]]=np.array(embedding_dict[i],dtype='float32')
        cand.append(embedding_matrix[word_dict[i][0]])
    cand=np.array(cand,dtype='float32')
    mu=np.mean(cand, axis=0)
    Sigma=np.cov(cand.T)
    norm=np.random.multivariate_normal(mu, Sigma, 1)
    for i in range(len(embedding_matrix)):
        if type(embedding_matrix[i])==int:
            embedding_matrix[i]=np.reshape(norm, 300)
    embedding_matrix[0]=np.zeros(300,dtype='float32')
    embedding_matrix=np.array(embedding_matrix,dtype='float32')
    print(embedding_matrix.shape)
    return embedding_matrix

In [ ]:
userid_dict,all_train_pn,all_label,all_train_id,all_test_pn,all_test_label,all_test_id,all_user_pos,all_test_user_pos=preprocess_user_file()

In [ ]:
word_dict,category,subcategory,news_words,news_body,news_v,news_sv,news_index=preprocess_news_file()

In [ ]:
embedding_mat=get_embedding(word_dict)

In [ ]:

def dcg_score(y_true, y_score, k=10):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)


def ndcg_score(y_true, y_score, k=10):
    best = dcg_score(y_true, y_true, k)
    actual = dcg_score(y_true, y_score, k)
    return actual / best


def mrr_score(y_true, y_score):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order)
    rr_score = y_true / (np.arange(len(y_true)) + 1)
    return np.sum(rr_score) / np.sum(y_true)


In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import keras
from keras.layers import *
from keras.models import Model
from keras import backend as K
from keras.optimizers import *

In [ ]:

def generate_batch_data_train(all_train_pn,all_label,all_train_id,batch_size):
    inputid = np.arange(len(all_label))
    np.random.shuffle(inputid)
    y=all_label
    batches = [inputid[range(batch_size*i, min(len(y), batch_size*(i+1)))] for i in range(len(y)//batch_size+1)]

    while (True):
        for i in batches:

            candidate = news_words[all_train_pn[i]]
            candidate_split=[candidate[:,k,:] for k in range(candidate.shape[1])]
            candidate_body = news_body[all_train_pn[i]]
            candidate_body_split=[candidate_body[:,k,:] for k in range(candidate_body.shape[1])]
            candidate_vertical = news_v[all_train_pn[i]]
            candidate_vertical_split=[candidate_vertical[:,k,:] for k in range(candidate_vertical.shape[1])]
            candidate_subvertical = news_sv[all_train_pn[i]]
            candidate_subvertical_split=[candidate_subvertical[:,k,:] for k in range(candidate_subvertical.shape[1])]
            
            browsed_news=news_words[all_user_pos[i]]
            browsed_news_split=[browsed_news[:,k,:] for k in range(browsed_news.shape[1])]
            browsed_news_body=news_body[all_user_pos[i]]
            browsed_news_body_split=[browsed_news_body[:,k,:] for k in range(browsed_news_body.shape[1])]
            browsed_news_vertical=news_v[all_user_pos[i]]
            browsed_news_vertical_split=[browsed_news_vertical[:,k,:] for k in range(browsed_news_vertical.shape[1])]
            browsed_news_subvertical=news_sv[all_user_pos[i]]
            browsed_news_subvertical_split=[browsed_news_subvertical[:,k,:] for k in range(browsed_news_subvertical.shape[1])]
            
        
            label=all_label[i]

            yield (candidate_split+browsed_news_split+candidate_body_split+browsed_news_body_split
                   +candidate_vertical_split+browsed_news_vertical_split +candidate_subvertical_split+browsed_news_subvertical_split, [label])

In [ ]:
def generate_batch_data_test(all_test_pn,all_label,all_test_id,batch_size):
    inputid = np.arange(len(all_label))
    y=all_label
    batches = [inputid[range(batch_size*i, min(len(y), batch_size*(i+1)))] for i in range(len(y)//batch_size+1)]

    while (True):
        for i in batches:
            candidate = news_words[all_test_pn[i]]
            candidate_body = news_body[all_test_pn[i]]
            candidate_vertical = news_v[all_test_pn[i]]
            candidate_subvertical = news_sv[all_test_pn[i]]

            browsed_news=news_words[all_test_user_pos[i]]
            browsed_news_split=[browsed_news[:,k,:] for k in range(browsed_news.shape[1])]
            browsed_news_body=news_body[all_test_user_pos[i]]
            browsed_news_body_split=[browsed_news_body[:,k,:] for k in range(browsed_news_body.shape[1])]
            browsed_news_vertical=news_v[all_test_user_pos[i]]
            browsed_news_vertical_split=[browsed_news_vertical[:,k,:] for k in range(browsed_news_vertical.shape[1])]
            browsed_news_subvertical=news_sv[all_test_user_pos[i]]
            browsed_news_subvertical_split=[browsed_news_subvertical[:,k,:] for k in range(browsed_news_subvertical.shape[1])]
            
        
            label=all_label[i]
            yield ([candidate]+ browsed_news_split+[candidate_body]+ browsed_news_body_split+[candidate_vertical]
                   + browsed_news_vertical_split+[candidate_subvertical]+ browsed_news_subvertical_split, [yy])

In [ ]:
import itertools
import keras
import random
results=[]
keras.backend.clear_session()

MAX_SENT_LENGTH=30
MAX_SENTS=50
npratio=4



MAX_BODY_LENGTH=300
title_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')

body_input = Input(shape=(MAX_BODY_LENGTH,), dtype='int32')
embedding_layer = Embedding(len(word_dict), 300, weights=[embedding_mat],trainable=True)

embedded_sequences_title = embedding_layer(title_input)
embedded_sequences_title=Dropout(0.2)(embedded_sequences_title)

embedded_sequences_body = embedding_layer(body_input)
embedded_sequences_body=Dropout(0.2)(embedded_sequences_body)

title_cnn = Convolution1D(nb_filter=400, filter_length=3,  padding='same', activation='relu', strides=1)(embedded_sequences_body)
title_cnn=Dropout(0.2)(title_cnn)

attention = Dense(200,activation='tanh')(title_cnn)
attention = Flatten()(Dense(1)(attention))
attention_weight = Activation('softmax')(attention)
title_rep=keras.layers.Dot((1, 1))([title_cnn, attention_weight])

body_cnn = Convolution1D(nb_filter=400, filter_length=3,  padding='same', activation='relu', strides=1)(embedded_sequences_body)
body_cnn=Dropout(0.2)(body_cnn)

attention_body = Dense(200,activation='tanh')(body_cnn)
attention_body = Flatten()(Dense(1)(attention_body))
attention_weight_body = Activation('softmax')(attention_body)
body_rep=keras.layers.Dot((1, 1))([body_cnn, attention_weight_body])

vinput=Input((1,), dtype='int32') 
svinput=Input((1,), dtype='int32') 
v_embedding_layer = Embedding(len(category)+1, 50,trainable=True)
sv_embedding_layer = Embedding(len(subcategory)+1, 50,trainable=True)
v_embedding=Dense(400,activation='relu')(Flatten()(v_embedding_layer(vinput)))
sv_embedding=Dense(400,activation='relu')(Flatten()(sv_embedding_layer(svinput)))

all_channel=[title_rep,body_rep,v_embedding,sv_embedding]
    
views=concatenate([Lambda(lambda x: K.expand_dims(x,axis=1))(channel) for channel in all_channel],axis=1)

attentionv = Dense(200,activation='tanh')(views)

attention_weightv =Lambda(lambda x:K.squeeze(x,axis=-1))(Dense(1)(attentionv))
attention_weightv =Activation('softmax')(attention_weightv)

newsrep=keras.layers.Dot((1, 1))([views, attention_weightv])

newsEncoder = Model([title_input,body_input,vinput,svinput],newsrep)

browsed_news_input = [keras.Input((MAX_SENT_LENGTH,), dtype='int32') for _ in range(MAX_SENTS)]
browsed_body_input = [keras.Input((MAX_BODY_LENGTH,), dtype='int32') for _ in range(MAX_SENTS)]

browsed_v_input = [keras.Input((1,), dtype='int32') for _ in range(MAX_SENTS)]
browsed_sv_input = [keras.Input((1,), dtype='int32') for _ in range(MAX_SENTS)]

browsednews = [newsEncoder([browsed_news_input[_],browsed_body_input[_],browsed_v_input[_],browsed_sv_input[_] ]) for _ in range(MAX_SENTS)]
browsednewsrep =concatenate([Lambda(lambda x: K.expand_dims(x,axis=1))(news) for news in browsednews],axis=1)    

attentionn = Dense(200,activation='tanh')(browsednewsrep)
attentionn =Flatten()(Dense(1)(attentionn))
attention_weightn = Activation('softmax')(attentionn)
user_rep=keras.layers.Dot((1, 1))([browsednewsrep, attention_weightn])

candidates_title = [keras.Input((MAX_SENT_LENGTH,), dtype='int32') for _ in range(1+npratio)]

candidates_body = [keras.Input((MAX_BODY_LENGTH,), dtype='int32') for _ in range(1+npratio)]

candidates_v = [keras.Input((1,), dtype='int32') for _ in range(1+npratio)]

candidates_sv = [keras.Input((1,), dtype='int32') for _ in range(1+npratio)]
candidate_vecs = [newsEncoder([candidates_title[_],candidates_body[_],candidates_v[_],candidates_sv[_]]) for _ in range(1+npratio)]

logits = [keras.layers.dot([user_rep, candidate_vec], axes=-1) for candidate_vec in candidate_vecs]
logits = keras.layers.Activation(keras.activations.softmax)(keras.layers.concatenate(logits))


model = Model(candidates_title+browsed_news_input+candidates_body+browsed_body_input+
              candidates_v+browsed_v_input+candidates_sv+browsed_sv_input, logits)


candidate_one_title = keras.Input((MAX_SENT_LENGTH,))

candidate_one_body = keras.Input((MAX_BODY_LENGTH,))

candidate_one_v = keras.Input((1,))

candidate_one_sv = keras.Input((1,))

candidate_one_vec=newsEncoder([candidate_one_title,candidate_one_body,candidate_one_v,candidate_one_sv])

score = keras.layers.Activation(keras.activations.sigmoid)(keras.layers.dot([user_rep, candidate_one_vec], axes=-1))
model_test = keras.Model([candidate_one_title]+browsed_news_input+[candidate_one_body] +browsed_body_input
                         +[candidate_one_v]+browsed_v_input+[candidate_one_sv]+browsed_sv_input, score)


model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['acc'])

for ep in range(3):
    traingen=generate_batch_data_train(all_train_pn,all_label,all_train_id, 30)
    model.fit_generator(traingen, epochs=1,steps_per_epoch=len(all_train_id)//30)
    testgen=generate_batch_data_test(all_test_pn,all_test_label,all_test_id, 30)
    click_score = model_test.predict_generator(testgen, steps=len(all_test_id)//30,verbose=1)
    from sklearn.metrics import roc_auc_score
    all_auc=[]
    all_mrr=[]
    all_ndcg=[]
    all_ndcg2=[]
    for m in all_test_index:
        if np.sum(all_test_label[m[0]:m[1]])!=0 and m[1]<len(click_score):
            all_auc.append(roc_auc_score(all_test_label[m[0]:m[1]],click_score[m[0]:m[1],0]))
            all_mrr.append(mrr_score(all_test_label[m[0]:m[1]],click_score[m[0]:m[1],0]))
            all_ndcg.append(ndcg_score(all_test_label[m[0]:m[1]],click_score[m[0]:m[1],0],k=5))
            all_ndcg2.append(ndcg_score(all_test_label[m[0]:m[1]],click_score[m[0]:m[1],0],k=10))
    results.append([np.mean(all_auc),np.mean(all_mrr),np.mean(all_ndcg),np.mean(all_ndcg2)])
    print(np.mean(all_auc),np.mean(all_mrr),np.mean(all_ndcg),np.mean(all_ndcg2))